In [6]:
import pandas as pd
import numpy
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
train = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\train.csv\train.csv")
test  = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\test.csv\test.csv")
train1 = train.drop(['Descript', 'Resolution', 'Address'], axis = 1)
test1 = test.drop(['Address'], axis = 1)

In [7]:
train1 = train1.query('Y != 90')

In [8]:
Dt = pd.to_datetime(train1['Dates'])
train1['Date'] = Dt.dt.date
train1['Year'] = Dt.dt.year
train1['Month'] = Dt.dt.month
train1['Day'] = Dt.dt.day
train1['Hour'] = Dt.dt.hour
train1.drop(["Dates"], axis = 1, inplace = True)

le = LabelEncoder()
train1['DayOfWeek'] = le.fit_transform(train1['DayOfWeek'])

count_encoding = train1['PdDistrict'].value_counts().to_dict()
train1['PdDistrict'] = train1['PdDistrict'].map(count_encoding)



train1['Category'] = le.fit_transform(train1['Category'])

train1.head()
# # DayOfWeek...label, PdDistrict...count

,Category,DayOfWeek,PdDistrict,X,Y,Date,Year,Month,Day,Hour
0,37,6,105284,-122.425892,37.774599,2015-05-13,2015,5,13,23
1,21,6,105284,-122.425892,37.774599,2015-05-13,2015,5,13,23
2,21,6,105284,-122.424363,37.800414,2015-05-13,2015,5,13,23
3,16,6,105284,-122.426995,37.800873,2015-05-13,2015,5,13,23
4,16,6,49311,-122.438738,37.771541,2015-05-13,2015,5,13,23


In [9]:
Dt = pd.to_datetime(test1['Dates'])
test1['Date'] = Dt.dt.date
test1['Year'] = Dt.dt.year
test1['Month'] = Dt.dt.month
test1['Day'] = Dt.dt.day
test1['Hour'] = Dt.dt.hour
test1.drop(["Dates"], axis = 1, inplace = True)

le = LabelEncoder()
test1['DayOfWeek'] = le.fit_transform(test1['DayOfWeek'])

count_encoding = test1['PdDistrict'].value_counts().to_dict()
test1['PdDistrict'] = test1['PdDistrict'].map(count_encoding)


test1.head()

,Id,DayOfWeek,PdDistrict,X,Y,Date,Year,Month,Day,Hour
0,0,3,89591,-122.399588,37.735051,2015-05-10,2015,5,10,23
1,1,3,89591,-122.391523,37.732432,2015-05-10,2015,5,10,23
2,2,3,107017,-122.426002,37.792212,2015-05-10,2015,5,10,23
3,3,3,80084,-122.437394,37.721412,2015-05-10,2015,5,10,23
4,4,3,80084,-122.437394,37.721412,2015-05-10,2015,5,10,23


In [10]:
cols = ['DayOfWeek', 'PdDistrict', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour']

X = train1[cols]
Y = train1['Category']

X_test = test1[cols]

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.3, random_state = 1219)

In [12]:
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

In [13]:
accuracy_score(y_val, y_pred)

0.28511551092465687

In [14]:
xgb_submission = model.predict(X_test)

In [15]:
import csv
pred = model.predict_proba(test1[['DayOfWeek', 'PdDistrict', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour']])
submission = pd.DataFrame(columns=['Id'], data=test)
submission = pd.concat([submission, pd.DataFrame(pred, columns=["ARSON", "ASSAULT", "BAD CHECKS", "BRIBERY", "BURGLARY", "DISORDERLY CONDUCT",
"DRIVING UNDER THE INFLUENCE", "DRUG/NARCOTIC", "DRUNKENNESS", "EMBEZZLEMENT", "EXTORTION",
"FAMILY OFFENSES", "FORGERY/COUNTERFEITING", "FRAUD", "GAMBLING", "KIDNAPPING", "LARCENY/THEFT",
"LIQUOR LAWS", "LOITERING", "MISSING PERSON", "NON-CRIMINAL", "OTHER OFFENSES", "PORNOGRAPHY/OBSCENE MAT",
"PROSTITUTION", "RECOVERED VEHICLE", "ROBBERY", "RUNAWAY", "SECONDARY CODES", "SEX OFFENSES FORCIBLE",
"SEX OFFENSES NON FORCIBLE", "STOLEN PROPERTY", "SUICIDE", "SUSPICIOUS OCC", "TREA", "TRESPASS",
"VANDALISM", "VEHICLE THEFT", "WARRANTS", "WEAPON LAWS"])], axis=1)

In [16]:
submission.to_csv('xgb_ad_drop.csv',index = False)